# Hyperparameter Tuning Guide

This notebook helps tune the model's hyperparameters using 2025 season data.

**Target:** Minimize MAE (Mean Absolute Error) < 2.5 positions

**Parameters to tune:**
- Qualifying weights (model vs practice)
- Race weights (grid, pace, tire, overtaking)
- DNF model parameters
- Bayesian ranking volatility

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import yaml

# Load current config
with open('../config/default.yaml') as f:
    config = yaml.safe_load(f)

print("Current Configuration:")
print(yaml.dump(config, default_flow_style=False))

## 1. Baseline: Current Performance

First, establish baseline MAE with current hyperparameters.

In [ ]:
# Load validation results
validation_file = Path('../outputs/validation_results.csv')

if validation_file.exists():
    df = pd.read_csv(validation_file)
    baseline_mae = df['mae'].mean()
    baseline_winner_acc = df['winner_correct'].mean() * 100
    
    print("📊 Baseline Performance (2025 season):")
    print(f"   MAE: {baseline_mae:.2f} positions")
    print(f"   Winner Accuracy: {baseline_winner_acc:.1f}%")
    print("   Target: MAE < 2.5")
else:
    print("⚠️ Run validation_report.ipynb first to generate baseline")
    baseline_mae = None

## 2. Race Simulation Weights

Test different weight combinations for race prediction.

**Current weights:**
- Grid: 30%
- Pace: 40%
- Tire Deg: 15%
- Overtaking: 15%

In [ ]:
# Define weight combinations to test
weight_configs = [
    {'grid': 0.30, 'pace': 0.40, 'tire': 0.15, 'overtaking': 0.15, 'name': 'Current'},
    {'grid': 0.35, 'pace': 0.35, 'tire': 0.15, 'overtaking': 0.15, 'name': 'More Grid'},
    {'grid': 0.25, 'pace': 0.45, 'tire': 0.15, 'overtaking': 0.15, 'name': 'More Pace'},
    {'grid': 0.30, 'pace': 0.35, 'tire': 0.20, 'overtaking': 0.15, 'name': 'More Tire'},
]

print("Weight Combinations to Test:")
for w in weight_configs:
    print(f"  {w['name']:15s}: Grid={w['grid']:.2f}, Pace={w['pace']:.2f}, Tire={w['tire']:.2f}, OT={w['overtaking']:.2f}")

## 3. Qualifying Blend Weights

Test different blends of model vs practice data.

**Hypothesis:** More practice weight = better predictions when practice data quality is high

In [ ]:
# Test different qualifying blends
quali_configs = [
    {'model': 0.40, 'fp3': 0.60, 'name': 'Current'},
    {'model': 0.30, 'fp3': 0.70, 'name': 'Trust Practice More'},
    {'model': 0.50, 'fp3': 0.50, 'name': 'Equal Weight'},
    {'model': 0.60, 'fp3': 0.40, 'name': 'Trust Model More'},
]

print("\nQualifying Blend Configurations:")
for q in quali_configs:
    print(f"  {q['name']:25s}: Model={q['model']:.0%}, FP3={q['fp3']:.0%}")

## 4. DNF Model Tuning

Adjust baseline DNF rates and team/driver penalties.

In [ ]:
dnf_configs = [
    {'base': 0.10, 'team': 0.05, 'driver': 0.03, 'name': 'Current'},
    {'base': 0.08, 'team': 0.04, 'driver': 0.02, 'name': 'Lower Risk'},
    {'base': 0.12, 'team': 0.06, 'driver': 0.04, 'name': 'Higher Risk'},
]

print("\nDNF Model Configurations:")
for d in dnf_configs:
    print(f"  {d['name']:15s}: Base={d['base']:.0%}, Team Penalty={d['team']:.0%}, Driver Penalty={d['driver']:.0%}")

## 5. Running Experiments

To test these configurations:

1. Update `config/default.yaml` with new parameters
2. Run `notebooks/validation_report.ipynb` to backtest
3. Compare MAE to baseline
4. Keep best-performing config

**Example:**
```yaml
race_simulation:
  grid_weight: 0.35        # Changed from 0.30
  pace_weight: 0.35        # Changed from 0.40
  tire_deg_weight: 0.15
  overtaking_weight: 0.15
```

## 6. Visualization: Parameter Impact

Plot how different parameters affect MAE.

In [ ]:
# Example: Impact of grid weight on MAE
# (Run validation with different configs and collect results)

# Placeholder data
grid_weights = [0.25, 0.30, 0.35, 0.40]
mae_scores = [2.4, 2.28, 2.5, 2.7]  # Replace with actual results

fig = go.Figure()

# Add MAE line
fig.add_trace(go.Scatter(
    x=grid_weights,
    y=mae_scores,
    mode='lines+markers',
    name='MAE',
    line=dict(width=2),
    marker=dict(size=8)
))

# Add target MAE line
fig.add_hline(
    y=2.5,
    line_dash="dash",
    line_color="red",
    annotation_text="Target MAE",
    annotation_position="right"
)

fig.update_layout(
    title='Impact of Grid Weight on Prediction Accuracy',
    xaxis_title='Grid Weight',
    yaxis_title='MAE (positions)',
    template='plotly_white',
    height=500,
    showlegend=True
)

fig.show()

print(f"\n✅ Optimal grid weight: {grid_weights[np.argmin(mae_scores)]:.2f}")

## 7. Best Practices

**When tuning hyperparameters:**
1. Change one parameter at a time
2. Use full 2025 season for validation (not just first 6 races)
3. Check both MAE and winner accuracy
4. Avoid overfitting to specific races
5. Document why you made the change

**Red flags:**
- MAE improves but winner accuracy drops → overfitting to midfield
- Extreme parameter values (> 0.50 for any single weight)
- Confidence scores become unrealistic (< 30% or > 95%)